## Goalseek in Actuarial Models with PyTorch

Goal seek is functionality in Excel that helps us solve equations. We can use the machinery from the deep learning framework PyTorch to implement goal seek in Python. We use a simplified version of the BasicTerm_M model for this demonstration.

In [3]:
import sys, os

if 'google.colab' in sys.modules:
    lib = 'basiclife'; lib_dir = '/content/'+ lib
    if not os.path.exists(lib_dir):
        !pip install lifelib
        import lifelib; lifelib.create(lib, lib_dir)
        
    %cd $lib_dir

In [4]:
!pip install torch

Now using node v18.19.0 (npm v10.2.3)

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Import and setup

The following cell contains all of the formulas and caching mechanisms, its contents are not the main focus so you can skip to the next cell if you like. Here is a brief overview of what the code does.

`Cash` is a cache to avoid a type of exponential runtime that is associated with sysems of recursive functions. `Ctx` is a class that contains any dependencies of a function that might change across runs of the model, in this case we change the premium as we optimize in search of the net premium.

Most of the code is formula definitions from the BasicTerm_M model.

In [22]:
from functools import wraps
from dataclasses import dataclass
from collections import defaultdict
import pandas as pd
import torch
from torch import optim

torch.set_default_dtype(torch.float64)

# constants
max_proj_len = 12 * 20 + 1

mp = pd.read_excel("BasicTerm_M/model_point_table.xlsx")
disc_rate = torch.tensor(pd.read_excel("BasicTerm_M/disc_rate_ann.xlsx")['zero_spot'].values)
mort_np = torch.tensor(pd.read_excel("BasicTerm_M/mort_table.xlsx").drop(columns=["Age"]).values)
sum_assured = torch.tensor(mp["sum_assured"].values)
issue_age = torch.tensor(mp["age_at_entry"].values)
policy_term = torch.tensor(mp["policy_term"].values)

# a cache decorator
class Cash:
    def __init__(self):
        self.reset()

    def reset(self):
        self.caches = defaultdict(dict)

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
            if key not in self.caches[func.__name__]:
                self.caches[func.__name__][key] = func(*args, **kwargs)
            return self.caches[func.__name__][key]

        return wrapper

cash = Cash()

# An object that contains context for the model
@dataclass(eq=False)
class Ctx:
    net_premium_pp: torch.Tensor

@cash
def get_annual_rate(duration):
    return mort_np[issue_age + duration - 18, min(duration, 5)]
@cash
def get_monthly_rate(duration):
    return 1 - (1 - get_annual_rate(duration)) ** (1/12)
@cash
def duration(t):
    return t // 12
@cash
def pols_death(t):
    return pols_if(t) * get_monthly_rate(duration(t))
@cash
def pols_if(t):
    if t == 0:
        return 1
    return pols_if(t - 1) - pols_lapse(t - 1) - pols_death(t - 1) - pols_maturity(t)
@cash
def lapse_rate(t):
    return max(0.1 - 0.02 * duration(t), 0.02)
@cash
def pols_lapse(t):
    return (pols_if(t) - pols_death(t)) * (1 - (1 - lapse_rate(t)) ** (1/12))
@cash
def pols_maturity(t):
    if t == 0:
        return 0
    return (t == 12 * policy_term) * (pols_if(t - 1) - pols_lapse(t - 1) - pols_death(t - 1))
@cash
def discount(t):
    return (1 + disc_rate[duration(t)]) ** (-t/12)
@cash
def claims(t):
    return pols_death(t) * sum_assured
@cash
def pv_pols_if():
    return sum(pols_if(t) * discount(t)  for t in range(max_proj_len))
@cash
def pv_claims():
    return sum(claims(t) * discount(t) for t in range(max_proj_len))
@cash
def net_premium_pp(ctx: Ctx):
    return ctx.net_premium_pp
@cash
def premiums(t, ctx):
    return net_premium_pp(ctx) * pols_if(t)
@cash
def pv_net_premiums(ctx):
    return sum(premiums(t, ctx) * discount(t) for t in range(max_proj_len))
@cash
def pv_net_cf(ctx):
    return pv_net_premiums(ctx) - pv_claims()

## Using PyTorch to find the net premium

Because the premium at which the net premiums are equal to claims is the net premium, we can minimize `torch.sum((pv_net_premiums(ctx) - pv_claims())**2)` and if this quantity approaches 0, we know that `pv_net_premiums = pv_claims` for each modelpoint.

In [21]:
ctx = Ctx(
    net_premium_pp=torch.zeros(len(mp), requires_grad=True)
)
optimizer = optim.LBFGS([ctx.net_premium_pp], lr=1)
def closure():
    cash.caches.clear()
    optimizer.zero_grad()
    loss = torch.sum((pv_net_premiums(ctx) - pv_claims()) ** 2)
    loss.backward()
    return loss.item()
optimizer.step(closure)
print(f"{pv_net_premiums(ctx) - pv_claims()=}")
print("It works!")

pv_net_premiums(ctx) - pv_claims()=tensor([-3.0353e-08,  5.1369e-08, -3.2707e-08,  ..., -6.0820e-08,
        -1.9495e-08, -5.3657e-08], grad_fn=<SubBackward0>)
It works!


The net premium can also be calculated as `pv_claims() / pv_pols_if()`. See that this yields the same results.

In [31]:
print(f"{ctx.net_premium_pp=}")
cash.reset()
print(f"{pv_claims() / pv_pols_if()=}")

ctx.net_premium_pp=tensor([ 63.2244,  40.7597, 105.7679,  ..., 105.0306,  27.5880,  21.2279],
       requires_grad=True)
pv_claims() / pv_pols_if()=tensor([ 63.2244,  40.7597, 105.7679,  ..., 105.0306,  27.5880,  21.2279])


## What's next

This goal seek relies on PyTorch's ability to minimize a loss function. Depending on how a loss function is defined, this technique could do things like setting premium jumps in term life insurance to optimize the profitability for a given lapse/mortality model.